In [21]:
import re, string, unicodedata
import nltk
import contractions # expanding contractions
import inflect # natural language related tasks of generating plurals, singular nouns, etc.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from pymongo import MongoClient
import string


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stagiaire2014\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
try:
    client = MongoClient('localhost', 27017) # host, port
except ConnectionFailure as e:
    sys.stderr.write("Could not connect to MongoDB: %s" % e)
    sys.exit(1)

# create database tweet_database
db = client.tweets_database

# delete collection
#db.tweets.drop()

# create collection tweets (eq. of a table in relational database)
tweets = db.tweets

In [23]:
# number of tweets
print("total number of tweets:", tweets.count())
print("number of english tweets:", tweets.find({'lang': 'en'}).count())

total number of tweets: 3364001
number of english tweets: 2706977


In [24]:
# import path to system only to be able to illustrate the examples in jupyter
import sys
path = 'D:\A_AHNE1\Tweet-Classification-Diabetes-Distress\preprocess'

if path not in sys.path:
    sys.path.insert(0, path)
sys.path

#sys.path.remove(path)
#sys.path

['D:\\A_AHNE1\\Tweet-Classification-Diabetes-Distress\\preprocess',
 '',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\python35.zip',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\DLLs',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages\\xgboost-0.7-py3.5.egg',
 'c:\\users\\stagiaire2014\\miniconda3\\envs\\deepscience\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Stagiaire2014\\.ipython']

In [25]:
# import preprocessing class 
from preprocess import Preprocess
prep = Preprocess()

In [30]:
# take first 1000 Tweets
count = 1
tweets_list = ["I love:D python 😄 :-)"] # add sample tweet to illustrate emoticons and emojis 
for tweet in tweets.find({'lang': 'en'}):
    
    tweet = prep.get_text(tweet)
    print("")
    print("")
    print("***ORIGINAL:")
    print(tweet)
    print("")
    
    tweet = prep.replace_contractions(tweet)
    tweet = prep.replace_hashtags_URL_USER(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    print("**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:")
    print(tweet)
    print("")
    
    tweet = prep.preprocess_emojis(tweet)
    tweet = prep.preprocess_emoticons(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.replace_numbers(tweet)
    print("**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:")
    print(tweet)
    print("")
    
    tweet = prep.remove_stopwords(tweet)
    tweet = prep.lemmatize_verbs(tweet)
    tweet = prep.stem_words(tweet)
    print("** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:")
    print(tweet)
    print("")
    
    tweets_list.append(tweet)
    #print(tweet)
    
    count += 1
    if (count == 1000):
        break



***ORIGINAL:
NEW! Reduce Age-Related Diseases! Linked to reducing cancer, diabetes, and cardiovascular disease. MORE INFO @… https://t.co/z2zdz1uYsd

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['NEW', 'Reduce', 'Age-Related', 'Diseases', 'Linked', 'to', 'reducing', 'cancer', 'diabetes', 'and', 'cardiovascular', 'disease', 'MORE', 'INFO', 'URL']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['new', 'reduce', 'age-related', 'diseases', 'linked', 'to', 'reducing', 'cancer', 'diabetes', 'and', 'cardiovascular', 'disease', 'more', 'info', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['new', 'reduc', 'age-rel', 'diseas', 'link', 'reduc', 'cancer', 'diabet', 'cardiovascular', 'diseas', 'info', 'URL']



***ORIGINAL:
RT @shanselman: Just got labs back from my 3 month #diabetes checkup, now 6 months on an Open Source "Artifical Pancreas." Hoping t… 

**REPLACED CONTRAC

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['USER', 'USER', 'USER', 'unbeat', 'type', 'two', 'diabet', 'cure', 'natur', 'home', 'URL']



***ORIGINAL:
RT @JakeAReid: 337,000 Nova Scotians live with diabetes or prediabetes. NS needs a provincial strategy: https://t.co/MjFulWavo2… 

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', '337,000', 'Nova', 'Scotians', 'live', 'with', 'diabetes', 'or', 'prediabetes', 'NS', 'needs', 'a', 'provincial', 'strategy', 'URL']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['rt', 'USER', '337,000', 'nova', 'scotians', 'live', 'with', 'diabetes', 'or', 'prediabetes', 'ns', 'needs', 'a', 'provincial', 'strategy', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', '337,000', 'nova', 'scotian', 'live', 'diabet', 'prediabet', 'ns', 'need', 'provinci', 'strategi', 'URL']



***ORIGINAL:
I have a nutrition ex



***ORIGINAL:
Oman wellness: Prevent diabetic heart disease https://t.co/kUSEm4t7Ds #heartdisease

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['Oman', 'wellness', 'Prevent', 'diabetic', 'heart', 'disease', 'URL', 'heartdisease']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['oman', 'wellness', 'prevent', 'diabetic', 'heart', 'disease', 'URL', 'heartdisease']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['oman', 'well', 'prevent', 'diabet', 'heart', 'diseas', 'URL', 'heartdiseas']



***ORIGINAL:
RT @drjanaway: As  a doctor I would say the biggest threat to our nations health isnt cancer, diabetes or dementia. 

Its @Conservatives…

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'As', 'a', 'doctor', 'I', 'would', 'say', 'the', 'biggest', 'threat', 'to', 'our', 'nations', 'health', 'is', 'not', 'can


** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'USER', 'vial', 'humalog', 'insulin', '275.50', 'oppos', 'two hundred and fifty-seven', 'two', 'week', 'ago', 'extra', 'fifty-four', 'per', 'month']



***ORIGINAL:
RT @simondankel: Important review of role of #insulin in #obesity. Time for a #paradigmshift! @whsource @davidludwigmd @FatEmperor https://…

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'Important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'Time', 'for', 'a', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['rt', 'USER', 'important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'time', 'for', 'a', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'import', 'review', 'role', 'insulin', 'obes', 

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['coach', 'diabet', 'take', 'med', 'natur', 'approach', 'listen', 'URL', 'jvcrum', 'healthi', 'market']



***ORIGINAL:
And we wonder about the epidemic of diabetes in this country... https://t.co/itKtxj2DJU

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['And', 'we', 'wonder', 'about', 'the', 'epidemic', 'of', 'diabetes', 'in', 'this', 'country', 'URL']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['and', 'we', 'wonder', 'about', 'the', 'epidemic', 'of', 'diabetes', 'in', 'this', 'country', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['wonder', 'epidem', 'diabet', 'countri', 'URL']



***ORIGINAL:
RT @UberFacts: If Americans stopped drinking soda every day, 2 million cases of diabetes could be prevented by 2020.

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS---------

...


**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['this', 'was', 'worth', 'the', 'risk', 'of', 'diabetes', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['worth', 'risk', 'diabet', 'URL']



***ORIGINAL:
RT @UberFacts: If Americans stopped drinking soda every day, 2 million cases of diabetes could be prevented by 2020.

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'If', 'Americans', 'stopped', 'drinking', 'soda', 'every', 'day', '2', 'million', 'cases', 'of', 'diabetes', 'could', 'be', 'prevented', 'by', '2020']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['rt', 'USER', 'if', 'americans', 'stopped', 'drinking', 'soda', 'every', 'day', 'two', 'million', 'cases', 'of', 'diabetes', 'could', 'be', 'prevented', 'by', 'two thousand and twenty']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'US


**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['diabetes', 'Group', 'Education', 'Curriculum', 'for', 'Older', 'Teens', 'With', 'Type', '1', 'Diabetes', 'URL']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['diabetes', 'group', 'education', 'curriculum', 'for', 'older', 'teens', 'with', 'type', 'one', 'diabetes', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['diabet', 'group', 'educ', 'curriculum', 'older', 'teen', 'type', 'one', 'diabet', 'URL']



***ORIGINAL:
Eat legumes to reduce your risk of type 2 #diabetestype2 #bloodsugar #insulinresistance https://t.co/i3DJzBuIrl https://t.co/RcppkDdcUE

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['Eat', 'legumes', 'to', 'reduce', 'your', 'risk', 'of', 'type', '2', 'diabetestype', '2', 'bloodsugar', 'insulinresistance', 'URL', 'URL']

**REPLACE EMOJIS AND EMOTICO

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['amazing', 'offer', 'major', 'discount', 'diabetes', 't1dlife', 'type', '1diabetes', 'gbdoc', 'USER', 'USER', 'URL']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['amaz', 'offer', 'major', 'discount', 'diabet', 't1dlife', 'type', '1diabet', 'gbdoc', 'USER', 'USER', 'URL']



***ORIGINAL:
RT @OriginateHealth: Previously unknown type of insulin-producing cell discovered
 https://t.co/3sMnyBgQ0B #healthinnovations #science… 

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'Previously', 'unknown', 'type', 'of', 'insulin-producing', 'cell', 'discovered', 'URL', 'healthinnovations', 'science']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['rt', 'USER', 'previously', 'unknown', 'type', 'of', 'insulin-producing', 'cell', 'discovered', 'URL', 'healthinnovations', 'science'


**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['rt', 'USER', 'diabetic', 'patients', 'in', 'yemen', 'struggle', 'as', 'insulin', 'prices', 'soar', 'today', 'who', 'delivered', '95,000', 'ampoules', 'of', 'insulin', 'to', 'the']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['rt', 'USER', 'diabet', 'patient', 'yemen', 'struggl', 'insulin', 'price', 'soar', 'today', 'deliv', '95,000', 'ampoul', 'insulin']



***ORIGINAL:
RT @prayerforce4: I pray against any affliction in your body including diabetes, cancer, heart disease, leukemia &amp; I declare you healed IJN…

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'I', 'pray', 'against', 'any', 'affliction', 'in', 'your', 'body', 'including', 'diabetes', 'cancer', 'heart', 'disease', 'leukemia', 'I', 'declare', 'you', 'healed', 'IJN']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-

['USER', 'America', 'looooooooves', 'rape', 'It', 'goes', 'rape', 'racism', 'sexism', 'football', 'and', 'diabetes', 'in', 'that', 'order', 'Loves', 'it']

**REPLACE EMOJIS AND EMOTICONS; REMOVE NON ASCII, TO LOWERCASE, REPLACE NUMBERS-------------:
['USER', 'america', 'looooooooves', 'rape', 'it', 'goes', 'rape', 'racism', 'sexism', 'football', 'and', 'diabetes', 'in', 'that', 'order', 'loves', 'it']

** REMOVE STOPWORDS, LEMMATIZE AND STEM-------------:
['USER', 'america', 'loooooooov', 'rape', 'go', 'rape', 'racism', 'sexism', 'footbal', 'diabet', 'order', 'love']



***ORIGINAL:
RT @KTLA: Long Beach girl needs help affording service dog after developing Type 1 diabetes, falling into coma https://t.co/KEF3VYlikj

**REPLACED CONTRACTIONS, HASHTAGS, URLs AND USER MENTIONS, TOKENISE; REMOVE PUNCTUATIONS-------------:
['RT', 'USER', 'Long', 'Beach', 'girl', 'needs', 'help', 'affording', 'service', 'dog', 'after', 'developing', 'Type', '1', 'diabetes', 'falling', 'into', 'coma', 'URL']



In [17]:

newlist = []
for tweet in tweets_list:
    print("")
    print("original:")
    print(tweet)
    tweet = prep.replace_contractions(tweet)
    tweet = prep.replace_hashtags_URL_USER(tweet)
    print("replaced contractions, hashtags, URLs and User mentions:")
    print(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    print("tokenise and remove punctuations:")
    print(tweet)
    tweet = prep.preprocess_emojis(tweet)
    tweet = prep.preprocess_emoticons(tweet)
    print("replace emojis and emoticons:")
    print(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.replace_numbers(tweet)
    print("remove non ascii, to lowercase, replace numbers:")
    print(tweet)
    tweet = prep.remove_stopwords(tweet)
    tweet = prep.lemmatize_verbs(tweet)
    tweet = prep.stem_words(tweet)
    print("remove stopwords, lemmatize and stem:")
    print(tweet)
    newlist.append(tweet)

    
tweets_list = newlist


original:
I love:D python 😄 :-)
replaced contractions, hashtags, URLs and User mentions:
I love:D python 😄 :-)
tokenise and remove punctuations:
['I', 'love', ':D', 'python', '😄', ':-)']
replace emojis and emoticons:
['I', 'love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']
remove non ascii, to lowercase, replace numbers:
['i', 'love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']
remove stopwords, lemmatize and stem:
['love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']
new:
['love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']

original:
NEW! Reduce Age-Related Diseases! Linked to reducing cancer, diabetes, and cardiovascular disease. MORE INFO @… https://t.co/z2zdz1uYsd
replaced contractions, hashtags, URLs and User mentions:
NEW! Reduce Age-Related Diseases! Linked to reducing cancer, diabetes, and cardiovascular disease. MORE INFO @… URL
tokenise and remove punctuations:
['NEW', 'Reduce', 'Age-Related', 'Diseases', 'Linked', 'to', 'reducing', 'cancer',

tokenise and remove punctuations:
['I', 'have', 'a', 'nutrition', 'exam', 'at', '2', 'and', 'at', '12.40', 'I', 'am', 'watching', 'my', 'lecture', 'on', 'diabetes', 'for', 'the', 'first', 'time']
replace emojis and emoticons:
['I', 'have', 'a', 'nutrition', 'exam', 'at', '2', 'and', 'at', '12.40', 'I', 'am', 'watching', 'my', 'lecture', 'on', 'diabetes', 'for', 'the', 'first', 'time']
remove non ascii, to lowercase, replace numbers:
['i', 'have', 'a', 'nutrition', 'exam', 'at', 'two', 'and', 'at', '12.40', 'i', 'am', 'watching', 'my', 'lecture', 'on', 'diabetes', 'for', 'the', 'first', 'time']
remove stopwords, lemmatize and stem:
['nutrit', 'exam', 'two', '12.40', 'watch', 'lectur', 'diabet', 'first', 'time']
new:
['nutrit', 'exam', 'two', '12.40', 'watch', 'lectur', 'diabet', 'first', 'time']

original:
this honestly makes me a bit ill, I can already taste the diabetes 😶 https://t.co/4Ig63dNe87
replaced contractions, hashtags, URLs and User mentions:
this honestly makes me a bit ill,

remove non ascii, to lowercase, replace numbers:
['rt', 'USER', 'news', 'research', 'from', 'USER', 'suggests', 'fatty', 'acids', 'consumed', 'during', 'breastfeeding', 'could', 'protect', 'against', 'type', '1diabetes', 'URL']
remove stopwords, lemmatize and stem:
['rt', 'USER', 'news', 'research', 'USER', 'suggest', 'fatti', 'acid', 'consum', 'breastfe', 'could', 'protect', 'type', '1diabet', 'URL']
new:
['rt', 'USER', 'news', 'research', 'USER', 'suggest', 'fatti', 'acid', 'consum', 'breastfe', 'could', 'protect', 'type', '1diabet', 'URL']

original:
Keep your #bloodsugar under control to help avoid these 8 #diabetes complications https://t.co/oS3miKc4Hk @usnews
replaced contractions, hashtags, URLs and User mentions:
Keep your bloodsugar under control to help avoid these 8 diabetes complications URL USER
tokenise and remove punctuations:
['Keep', 'your', 'bloodsugar', 'under', 'control', 'to', 'help', 'avoid', 'these', '8', 'diabetes', 'complications', 'URL', 'USER']
replace emojis

remove stopwords, lemmatize and stem:
['rt', 'USER', 'USER', 'vial', 'humalog', 'insulin', '275.50', 'oppos', 'two hundred and fifty-seven', 'two', 'week', 'ago', 'extra', 'fifty-four', 'per', 'month']
new:
['rt', 'USER', 'USER', 'vial', 'humalog', 'insulin', '275.50', 'oppos', 'two hundred and fifty-seven', 'two', 'week', 'ago', 'extra', 'fifty-four', 'per', 'month']

original:
RT @simondankel: Important review of role of #insulin in #obesity. Time for a #paradigmshift! @whsource @davidludwigmd @FatEmperor https://…
replaced contractions, hashtags, URLs and User mentions:
RT USER: Important review of role of insulin in obesity. Time for a paradigmshift! USER USER USER URL
tokenise and remove punctuations:
['RT', 'USER', 'Important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'Time', 'for', 'a', 'paradigmshift', 'USER', 'USER', 'USER', 'URL']
replace emojis and emoticons:
['RT', 'USER', 'Important', 'review', 'of', 'role', 'of', 'insulin', 'in', 'obesity', 'Time', 'for',

['USER', 'USER', 'If', 'my', 'number', 'is', 'called', 'one', 'day', 'because', 'of', 'lack', 'of', 'insulin', 'access', 'I', 'will', 'gladly', 'do', 'th', 'URL']
replace emojis and emoticons:
['USER', 'USER', 'If', 'my', 'number', 'is', 'called', 'one', 'day', 'because', 'of', 'lack', 'of', 'insulin', 'access', 'I', 'will', 'gladly', 'do', 'th', 'URL']
remove non ascii, to lowercase, replace numbers:
['USER', 'USER', 'if', 'my', 'number', 'is', 'called', 'one', 'day', 'because', 'of', 'lack', 'of', 'insulin', 'access', 'i', 'will', 'gladly', 'do', 'th', 'URL']
remove stopwords, lemmatize and stem:
['USER', 'USER', 'number', 'call', 'one', 'day', 'lack', 'insulin', 'access', 'glad', 'th', 'URL']
new:
['USER', 'USER', 'number', 'call', 'one', 'day', 'lack', 'insulin', 'access', 'glad', 'th', 'URL']

original:
RT @UberFacts: If Americans stopped drinking soda every day, 2 million cases of diabetes could be prevented by 2020.
replaced contractions, hashtags, URLs and User mentions:
RT USE

...

replace emojis and emoticons:
['Seven', 'yoga', 'poses', 'for', 'diabetes', 'URL', 'Louisville', 'hypnosis', 'weight', 'loss', 'clinic', 'weight', 'loss', 'Louisvi', 'URL']
remove non ascii, to lowercase, replace numbers:
['seven', 'yoga', 'poses', 'for', 'diabetes', 'URL', 'louisville', 'hypnosis', 'weight', 'loss', 'clinic', 'weight', 'loss', 'louisvi', 'URL']
remove stopwords, lemmatize and stem:
['seven', 'yoga', 'pose', 'diabet', 'URL', 'louisvill', 'hypnosi', 'weight', 'loss', 'clinic', 'weight', 'loss', 'louisvi', 'URL']
new:
['seven', 'yoga', 'pose', 'diabet', 'URL', 'louisvill', 'hypnosi', 'weight', 'loss', 'clinic', 'weight', 'loss', 'louisvi', 'URL']

original:
.@LillyDiabetes dying. You should be ashamed. #insulin4all
replaced contractions, hashtags, URLs and User mentions:
.USER dying. You should be ashamed. insulin4all
tokenise and remove punctuations:
['USER', 'dying', 'You', 'should', 'be', 'ashamed', 'insulin', '4all']
replace emojis and emoticons:
['USER', 'dying', 'Y

['rt', 'USER', 'if', 'americans', 'stopped', 'drinking', 'soda', 'every', 'day', 'two', 'million', 'cases', 'of', 'diabetes', 'could', 'be', 'prevented', 'by', 'two thousand and twenty']
remove stopwords, lemmatize and stem:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
new:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']

original:
RT @mnt: Seven yoga poses for diabetes https://t.co/sWmOHrAH4K https://t.co/RR0BHexur3
replaced contractions, hashtags, URLs and User mentions:
RT USER: Seven yoga poses for diabetes URL URL
tokenise and remove punctuations:
['RT', 'USER', 'Seven', 'yoga', 'poses', 'for', 'diabetes', 'URL', 'URL']
replace emojis and emoticons:
['RT', 'USER', 'Seven', 'yoga', 'poses', 'for', 'diabetes', 'URL', 'URL']
remove non ascii, to lowercase, replace numbers:
['rt'

original:
RT @maaaaaadiison: then why the fuck do u guys keep raising the prices of insulin
https://t.co/25YNAgN2yK
replaced contractions, hashtags, URLs and User mentions:
RT USER: then why the fuck do you guys keep raising the prices of insulin
URL
tokenise and remove punctuations:
['RT', 'USER', 'then', 'why', 'the', 'fuck', 'do', 'you', 'guys', 'keep', 'raising', 'the', 'prices', 'of', 'insulin', 'URL']
replace emojis and emoticons:
['RT', 'USER', 'then', 'why', 'the', 'fuck', 'do', 'you', 'guys', 'keep', 'raising', 'the', 'prices', 'of', 'insulin', 'URL']
remove non ascii, to lowercase, replace numbers:
['rt', 'USER', 'then', 'why', 'the', 'fuck', 'do', 'you', 'guys', 'keep', 'raising', 'the', 'prices', 'of', 'insulin', 'URL']
remove stopwords, lemmatize and stem:
['rt', 'USER', 'fuck', 'guy', 'keep', 'rais', 'price', 'insulin', 'URL']
new:
['rt', 'USER', 'fuck', 'guy', 'keep', 'rais', 'price', 'insulin', 'URL']

original:
RT @bigfatsurprise: I signed this, bc we need evidence-bas

['people', 'who', 'make', 'the', 'common', 'diabetes', 'joke', 'on', 'a', 'food', 'item', 'that', 'contains', 'a', 'lot', 'of', 'sugar', 'do', 'not']
remove non ascii, to lowercase, replace numbers:
['people', 'who', 'make', 'the', 'common', 'diabetes', 'joke', 'on', 'a', 'food', 'item', 'that', 'contains', 'a', 'lot', 'of', 'sugar', 'do', 'not']
remove stopwords, lemmatize and stem:
['peopl', 'make', 'common', 'diabet', 'joke', 'food', 'item', 'contain', 'lot', 'sugar']
new:
['peopl', 'make', 'common', 'diabet', 'joke', 'food', 'item', 'contain', 'lot', 'sugar']

original:
RT @prayerforce4: I pray against any affliction in your body including diabetes, cancer, heart disease, leukemia &amp; I declare you healed IJN…
replaced contractions, hashtags, URLs and User mentions:
RT USER: I pray against any affliction in your body including diabetes, cancer, heart disease, leukemia &amp; I declare you healed IJN…
tokenise and remove punctuations:
['RT', 'USER', 'I', 'pray', 'against', 'any', '

remove stopwords, lemmatize and stem:
['rt', 'USER', 'bpa', 'mimic', 'hormon', 'estrogen', 'link', 'breast', 'cancer', 'diabet', 'preterm', 'birth', 'URL']
new:
['rt', 'USER', 'bpa', 'mimic', 'hormon', 'estrogen', 'link', 'breast', 'cancer', 'diabet', 'preterm', 'birth', 'URL']

original:
RT @1004project_: I highly suggest to eat first before doing a mental task. Our brains need glucose to function well. Like this Lolol… 
replaced contractions, hashtags, URLs and User mentions:
RT USER: I highly suggest to eat first before doing a mental task. Our brains need glucose to function well. Like this Lolol… 
tokenise and remove punctuations:
['RT', 'USER', 'I', 'highly', 'suggest', 'to', 'eat', 'first', 'before', 'doing', 'a', 'mental', 'task', 'Our', 'brains', 'need', 'glucose', 'to', 'function', 'well', 'Like', 'this', 'Lolol']
replace emojis and emoticons:
['RT', 'USER', 'I', 'highly', 'suggest', 'to', 'eat', 'first', 'before', 'doing', 'a', 'mental', 'task', 'Our', 'brains', 'need', 'glu

In [11]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
from defines import Grammar
from nltk.stem import LancasterStemmer, PorterStemmer, SnowballStemmer, WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer

class Grammar1:
    STOPWORDS = stopwords.words('english')
    STEMMER_LANCASTER = LancasterStemmer() # aggressive, fast, sometimes confusing
    STEMMER_PORTER = PorterStemmer(mode='NLTK_EXTENSIONS') # mode that includes further improvements
    STEMMER_SNOWBALL = SnowballStemmer("english") # improved porter

Constant_words = ['URL', 'USER', 'EMOT_SMILE', 'EMOT_LAUGH', 'EMOT_LOVE', 'EMOT_WINK',
                  'EMOT_SAD_ANGRY', 'EMOT_SURPRISED', 'EMOT_CRY']
    
def stem_words(tweet, stemmer):
    """ Stem words in list of tokenized words

        Parameter:
            - tweet :   tokenized list of words of the tweet
            - stemmer : algorithm to use for stemming
                        - Grammar.STEMMER_SNOWBALL (default)
                        - Grammar.PORTER
                        - Grammar.STEMMER_LANCASTER


        Remark: Three major stemming algorithms
            - Porter: most commonly used, oldest, most computationally expensive
            - Snowball / Porter2: better than Porter, a bit faster than Porter
            - Lancaster: aggressive algorithm, sometimes to a fault; fastest algo
                        often not intuitiive words; reduces words space hugely
    """
    for ind, word in enumerate(tweet):
        if word not in Constant_words: # do not change words like USER, URL, EMOT_SMILE,...
            tweet[ind] = stemmer.stem(word)
    return tweet

newlist = []
tweet_list1 = tweets_list
tweet_list2 = tweets_list
tweet_list3 = tweets_list
for i,tweet in enumerate(tweets_list):
    #print("")
    print("old:")
    print(tweet)
    tweet1 = tweet_list1[i]
    tweet2 = tweet_list2[i]
    tweet3 = tweet_list3[i]
    tweet1 = stem_words(tweet1, Grammar1.STEMMER_SNOWBALL)
    #tweet2 = stem_words(tweet2, Grammar1.STEMMER_LANCASTER)
    #tweet3 = stem_words(tweet3, Grammar1.STEMMER_PORTER)
    print("new")
    print(tweet1)
    if tweet1 != tweet2 or tweet1 != tweet3 or tweet2 != tweet3:
        print("")
        print("old:")
        print(tweet)
        print("SNOWBALL:")
        print(tweet1)
        print("LANCASTER:")
        print(tweet2)
        print("PORTER:")
        print(tweet3)
    #newlist.append(tweet)
    #print("new:")
    #print(tweet)
#tweets_list = newlist

old:
['love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']
new
['love', 'EMOT_LAUGH', 'python', 'EMOT_LAUGH', 'EMOT_SMILE']
old:
['new', 'reduc', 'age-rel', 'diseas', 'link', 'reduc', 'cancer', 'diabet', 'cardiovascular', 'diseas', 'info', 'URL']
new
['new', 'reduc', 'age-rel', 'disea', 'link', 'reduc', 'cancer', 'diabet', 'cardiovascular', 'disea', 'info', 'URL']
old:
['rt', 'USER', 'get', 'lab', 'back', 'three', 'month', 'diabet', 'checkup', 'six', 'month', 'open', 'sourc', 'artif', 'pancrea', 'hop']
new
['rt', 'USER', 'get', 'lab', 'back', 'three', 'month', 'diabet', 'checkup', 'six', 'month', 'open', 'sourc', 'artif', 'pancrea', 'hop']
old:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
new
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
old:
['im', 'idiot', 'buy', 'sandw

new
['rt', 'USER', 'disea', 'like', 'obe', 'diabet', 'heart', 'disea', 'requir', 'everyday', 'use', 'med', 'well', 'prevent', 'healthi', 'di']
old:
['rt', 'USER', 'high', 'suggest', 'eat', 'first', 'mental', 'task', 'brain', 'need', 'glucos', 'function', 'well', 'like', 'lolol']
new
['rt', 'USER', 'high', 'suggest', 'eat', 'first', 'mental', 'task', 'brain', 'need', 'gluco', 'function', 'well', 'like', 'lolol']
old:
['USER', 'a1', 'lower', 'risk', 'heart', 'diseas', 'benefit', 'insulin', 'level', 'also', 'contain', 'vitamin', 'URL']
new
['USER', 'a1', 'lower', 'risk', 'heart', 'disea', 'benefit', 'insulin', 'level', 'also', 'contain', 'vitamin', 'URL']
old:
['rt', 'USER', 'high', 'suggest', 'eat', 'first', 'mental', 'task', 'brain', 'need', 'glucos', 'function', 'well', 'like', 'lolol']
new
['rt', 'USER', 'high', 'suggest', 'eat', 'first', 'mental', 'task', 'brain', 'need', 'gluco', 'function', 'well', 'like', 'lolol']
old:
['twelv', 'percent', 'death', 'caus', 'diabet', 'higher', 'tru

['rt', 'USER', 'carb', 'count', 'one', 'form', 'meal', 'plan', 'help', 'peopl', 'diabet', 'manag', 'blood', 'sugar', 'level']
old:
['USER', 'ceo', 'yuck', 'morn', 'show', 'say', 'lead', 'patient', 'control', 'diabet', 'bet', 'URL']
new
['USER', 'ceo', 'yuck', 'morn', 'show', 'say', 'lead', 'patient', 'control', 'diabet', 'bet', 'URL']
old:
['wearabl', 'sweat', 'sensor', 'monitor', 'blood', 'glucos', 'may', 'help', 'diagnos', 'cysticfibrosi', 'URL', 'proof', 'concept', 'studi']
new
['wearabl', 'sweat', 'sensor', 'monitor', 'blood', 'gluco', 'may', 'help', 'diagno', 'cysticfibrosi', 'URL', 'proof', 'concept', 'studi']
old:
['rt', 'USER', 'genius', 'one hundr', 'cal', 'snack', 'pack', 'eat', 'small', 'amount', 'carb', 'increas', 'blood', 'sugar', 'insulin', 'level', 'result']
new
['rt', 'USER', 'genius', 'one hundr', 'cal', 'snack', 'pack', 'eat', 'small', 'amount', 'carb', 'increa', 'blood', 'sugar', 'insulin', 'level', 'result']
old:
['resourc', 'clinician', 'children', 'young', 'peopl'

old:
['USER', 'look', 'mrs', 'pleas', 'EMOT_SMILE', 'URL']
new
['USER', 'look', 'mrs', 'plea', 'EMOT_SMILE', 'URL']
old:
['clinton', 'say', 'epidem', 'level', 'diabet', 'phasummit', 'URL']
new
['clinton', 'say', 'epidem', 'level', 'diabet', 'phasummit', 'URL']
old:
['novo', 'nordisk', 'bet', 'new', 'obes', 'drug', 'recip', 'london', 'reuter', 'hit', 'price', 'pressur', 'core', 'insulin', 'URL']
new
['novo', 'nordisk', 'bet', 'new', 'obe', 'drug', 'recip', 'london', 'reuter', 'hit', 'price', 'pressur', 'core', 'insulin', 'URL']
old:
['attenu', 'diabet', 'nephropathi', 'sanziguben', 'granul', 'inhibit', 'emt', 'nrf', 'two', 'mediat', 'anti-oxid', 'eff', 'URL']
new
['attenu', 'diabet', 'nephropathi', 'sanziguben', 'granul', 'inhibit', 'emt', 'nrf', 'two', 'mediat', 'anti-oxid', 'eff', 'URL']
old:
['may', 'increas', 'risk', 'diseas', 'inadequ', 'sleep', 'associ', 'obes', 'diabet', 'stroke', 'heart', 'diseas', 'cancer']
new
['may', 'increa', 'risk', 'disea', 'inadequ', 'sleep', 'associ', 'o

new
['rt', 'USER', 'get', 'lab', 'back', 'three', 'month', 'diabet', 'checkup', 'six', 'month', 'open', 'sourc', 'artif', 'pancrea', 'hop']
old:
['rt', 'USER', 'much', 'control', 'plwd', 'type', 'one', 'glucos', 'even', 'low', 'a1c', 'sixti', 'target', 'mark', 'davi']
new
['rt', 'USER', 'much', 'control', 'plwd', 'type', 'one', 'gluco', 'even', 'low', 'a1c', 'sixti', 'target', 'mark', 'davi']
old:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
new
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
old:
['USER', 'sudden', 'type', 'two', 'diabet']
new
['USER', 'sudden', 'type', 'two', 'diabet']
old:
['rt', 'USER', 'american', 'stop', 'drink', 'soda', 'everi', 'day', 'two', 'million', 'case', 'diabet', 'could', 'prevent', 'two thousand and twenti']
new
['rt', 'USER', 'american', 'stop', '

In [49]:
# additional text cleaning

# Locate and correct common typos and misspellings
#Handling numbers and dates inside the text



In [50]:
prep.Constant_words

['URL',
 'USER',
 'EMOT_SMILE',
 'EMOT_LAUGH',
 'EMOT_LOVE',
 'EMOT_WINK',
 'EMOT_SAD_ANGRY',
 'EMOT_SURPRISED',
 'EMOT_CRY']